
# Case Study: Leveraging Web Scraping for Financial Data Acquisition

Lecture video: https://drive.google.com/file/d/10D5BeK3TeQqFhT7YBnlCJtF4zAdWtExq/view?usp=sharing


## Introduction: Cutting Corners or Cutting Costs? The Finviz Data Extraction Dilemma

In the fast-paced world of finance, data is the lifeblood that drives decision-making, from high-frequency trading to long-term investment strategies. Premium data vendors like Reuters and Bloomberg have dominated this space, charging hefty sums for access to their treasure troves of financial data. However, with the rise of the digital age, alternative data sources have emerged. Websites such as finviz.com offer a plethora of financial data for free, prompting many to question the value of expensive subscriptions.

In this case study, we dive into a scenario at a mid-sized investment firm. The CFO, looking to optimize costs, stumbles upon finviz.com's stock screener. He wonders: Can the firm develop a web parser to extract all this free data, thereby potentially saving thousands of dollars in subscription fees? As the firm's lead data scientist, the responsibility falls on you to evaluate this proposition.

You'll need to consider not only the technical challenges but also the ethical and business implications. Would the quality of free data match that of premium vendors? Is it ethical, or even legal, to scrape data from a website without permission? And what would be the long-term consequences for the firm's reputation and bottom line?

Through this case study, students will grapple with the challenges of data acquisition in the digital age, the ethics of web scraping, and the broader implications of business decisions in the tech-driven world of modern finance.



Classification:
* Web Data Extraction

Motivation:
* Cutting Costs in Financial Data Procurement

Intent:
* To create a web parser that extracts stock data from finviz.com's stock screener to replace the data subscriptions from expensive vendors like Reuters and Bloomberg, thereby saving significant costs.
  * Get all the tickers (20/page, get all pages)
  * Get all columns (not just overview tab)
  * Handle messy data (Clean/Transform data)
  * Self-healing (if webpage returns error, back off and try again later)
  * Cache data (so we don't keep pinging the webpage)


Examples:

* A hedge fund previously relying on premium financial data subscriptions notices that much of the data they use is available on free stock screeners online.
* An academic researcher looking to perform a study on stock market trends but has a limited grant budget. The researcher decides to scrape publicly available data instead of purchasing datasets.
Consequences:

Positive Consequences:
* Cost Savings: By using a web parser, the firm can save a substantial amount of money by avoiding premium subscription fees.
* Data Customization: The firm has more control over the data it gathers and can tweak the parser to fit specific data needs.

Negative Consequences:
* Legal & Ethical Implications: Web scraping can infringe on the terms of service of a website. Sites like finviz.com might have stipulations against automated data extraction.
* Data Reliability & Quality: Data from free sources might not always be as accurate or reliable as premium data sources. There's a potential risk associated with using potentially less reliable data.
* Maintenance Overhead: Websites can change their structure. If finviz.com changes its layout, the parser might stop working, and it will need regular updates.

In [ ]:
!pip install random_user_agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.8 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import time
from functools import cache

# Set up random user agent
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)




def getHeader():
    return {
        "User-Agent": user_agent_rotator.get_random_user_agent(),
    }

@cache
def getPage(url):
    response = requests.get(url, headers=getHeader())
    if response.status_code != 200:
        print(f'Error {url}')
        time.sleep(1)
        response = requests.get(url, headers=getHeader())
        if response.status_code != 200:
            print(f'2nd Error {url}')
            time.sleep(1)
            response = requests.get(url, headers=getHeader())
            if response.status_code != 200:
                print(f'3rd Error {url}')
                errors.append(url)
                raise Exception

    print(f'Success {url}')
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


dfs=[]
errors=[]
for r in range(1,12000,20):
    url=f'https://finviz.com/screener.ashx?v=152&c={",".join([str(x) for x in range(100)])}&r={r}'

    soup=getPage(url)
    table = soup.find("table",
                            attrs={'class':'styled-table-new is-rounded is-tabular-nums w-full screener_table'}
                            )
    header=[cell.text.strip() for cell in table.find_all('tr')[0].find_all('th')]
    data=[[cell.text.strip() for cell in row.find_all('td')] for row in table.find_all('tr')[1:]]
    dfpage=pd.DataFrame(data,columns=header)
    dfpage=dfpage.drop(columns='No.').drop_duplicates('Ticker').set_index('Ticker')
    dfs.append(dfpage)
    if len(dfpage)<20:
        break
finvizDf=pd.concat(dfs)
finvizDf=finvizDf.loc[~finvizDf.index.duplicated()]
if len(finvizDf)<8000:
    error



Success https://finviz.com/screener.ashx?v=152&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99&r=1
Success https://finviz.com/screener.ashx?v=152&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99&r=21
Success https://finviz.com/screener.ashx?v=152&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,9

In [ ]:
# remove duplicate row
finvizDf=finvizDf.drop_duplicates(subset='Ticker').set_index('Ticker')

KeyError: ignored

In [ ]:
finvizDf.columns.value_counts() # duplicate column

Company          1
52W High         1
Price            1
Rel Volume       1
Avg Volume       1
                ..
Inst Trans       1
Inst Own         1
Insider Trans    1
Insider Own      1
Low              1
Length: 87, dtype: int64

In [ ]:
cols=[]
for idx,c in enumerate(finvizDf.columns):
  if c not in cols:
    cols.append(c)
  else:
    if '%' in finvizDf.iloc[0,idx]:
      cols.append(c+'%')
    else:
      cols.append(c+'$')
finvizDf.columns=cols


In [ ]:
# Duplicate col handled
finvizDf.filter(like='Divi')

,Dividend,Dividend$
Ticker,,
A,0.80%,0.90
AA,1.23%,0.33
AAAU,-,-
AAC,-,-
AACG,-,-
...,...,...
ZVSA,-,-
ZWS,0.83%,0.23
ZYME,-,-


In [ ]:
finvizDf.to_parquet('finviz.par')

